In [1]:
import pandas as pd
from random import shuffle
import numpy as np
import pickle

In [2]:
df_ch=pd.read_csv("challenges.csv")
df_sub=pd.read_csv("submissions.csv")

In [3]:
print df_ch.columns
print df_sub.columns

Index([u'challenge_id', u'contest_id', u'domain', u'subdomain', u'difficulty',
       u'solved_submission_count', u'total_submissions_count'],
      dtype='object')
Index([u'hacker_id', u'contest_id', u'challenge_id', u'language', u'solved',
       u'created_at'],
      dtype='object')


In [4]:
#(challenge_id,contest_id) is pk
df = pd.merge(df_ch, df_sub, how='inner', on=['challenge_id', 'contest_id'])

In [ ]:
df=df.reindex(np.random.permutation(df.index))

In [5]:
print df.columns

Index([u'challenge_id', u'contest_id', u'domain', u'subdomain', u'difficulty',
       u'solved_submission_count', u'total_submissions_count', u'hacker_id',
       u'language', u'solved', u'created_at'],
      dtype='object')


In [6]:
cols=['challenge_id', 'contest_id', 'domain', 'subdomain', 'difficulty',
       'solved_submission_count', 'total_submissions_count', 'hacker_id',
       'language', 'solved', 'created_at']

In [7]:
for col in cols:
    #df[col]=df[col].fillna(df[col].value_counts().index[0])
    df[col]=df[col].fillna(method='ffill')
    #print col,sum(df[col].isnull())

In [8]:
idxmap={}
idx=0
for col in df['subdomain'].unique():
    idxmap[idx]=col
    idx+=1

In [9]:
"""sdf=df.groupby(['hacker_id','subdomain'])
sdf_desc=sdf['solved'].describe()
print sdf_desc['00004cf8b853ad0d']['Bit Manipulation']['count']"""

"sdf=df.groupby(['hacker_id','subdomain'])\nsdf_desc=sdf['solved'].describe()\nprint sdf_desc['00004cf8b853ad0d']['Bit Manipulation']['count']"

In [10]:
"""hpref={}
for hacker in df['hacker_id'].unique():
    if hacker not in hpref:
        hpref[hacker]=[0]*num_sub
    for sub in subdom:
        if 'count' in sdf_desc[hacker][sub]:
            hpref[hacker][idxmap[sub]]=sdf_desc[hacker][sub]['count']"""

"hpref={}\nfor hacker in df['hacker_id'].unique():\n    if hacker not in hpref:\n        hpref[hacker]=[0]*num_sub\n    for sub in subdom:\n        if 'count' in sdf_desc[hacker][sub]:\n            hpref[hacker][idxmap[sub]]=sdf_desc[hacker][sub]['count']"

In [11]:

def save_object(obj, filename):
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
    return

def load_object(filename):
    with open(filename, 'rb') as input:
        obj=pickle.load(input)
    return obj

In [12]:
"""save_object(hpref,"hpref.pkl")
save_object(sdf_desc,"sdf_desc.pkl")
save_object(sdf,"sdf.pkl")"""

'save_object(hpref,"hpref.pkl")\nsave_object(sdf_desc,"sdf_desc.pkl")\nsave_object(sdf,"sdf.pkl")'

In [13]:
hpref=load_object("hpref.pkl")
sdf_desc=load_object("sdf_desc.pkl")
sdf=load_object("sdf.pkl")

In [14]:
mode_sub_dom=df["subdomain"].value_counts().index[0]
print mode_sub_dom

Introduction


In [16]:
df.sort(['solved_submission_count','total_submissions_count'], ascending=[False, False], inplace=True)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [17]:
subdom={}
for col in df['subdomain'].unique():
    subdom[col]=df.loc[df['subdomain']==col]
    

In [18]:
"""hacker_solved={}
#hackers=df['hacker_id'].unique()
for i,row in df.iterrows():
    if row['solved']==1:
        if row['hacker_id'] not in hacker_solved:
            hacker_solved[row['hacker_id']]=set()
        hacker_solved[row['hacker_id']].add(row['challenge_id'])"""

"hacker_solved={}\n#hackers=df['hacker_id'].unique()\nfor i,row in df.iterrows():\n    if row['solved']==1:\n        if row['hacker_id'] not in hacker_solved:\n            hacker_solved[row['hacker_id']]=set()\n        hacker_solved[row['hacker_id']].add(row['challenge_id'])"

In [19]:
#save_object(hacker_solved,"hacker_solved.pkl")
hacker_solved=load_object("hacker_solved.pkl")
print len(hacker_solved)

9289


In [20]:
#print subdom['Dynamic Programming'].head()

In [21]:
l=hpref["00004cf8b853ad0d"]
print idxmap[l.index(max(l))] #sorting

Sorting


In [22]:
#to do : before adding challenge to set check whether hacker has done it
rec={}
iter=0
hackers=df['hacker_id'].unique()
#hackers=["00004cf8b853ad0d","3827969344861ac8","b630307ea7151c3a"]
for hacker in hackers:
    print "\ruser "+str(iter)+":"+hacker,
    l=hpref[hacker]
    pref=[]
    for i in range(len(l)) :
        #if l[i]!=0:
        pref.append((l[i],i))
    pref.sort(reverse=True)
    hrec=set()
    size=3
    num=12
    for t in pref:
        tdf=subdom[idxmap[t[1]]]
        if len(hrec)>=num:
                break
        count=0
        for ii,row in tdf.iterrows():
            if ((row['hacker_id']!=hacker) or (row['hacker_id']==hacker and row['solved']==0)) and row['contest_id']=='c8ff662c97d345d2':
                if row['challenge_id'] not in hrec:
                    if hacker not in hacker_solved:
                        hrec.add(row['challenge_id'])
                        ss=set()
                        ss.add(row['challenge_id'])
                        hacker_solved[hacker]=ss
                        count+=1
                    elif row['challenge_id'] not in hacker_solved[hacker]:
                        hrec.add(row['challenge_id'])
                        hacker_solved[hacker].add(row['challenge_id'])
                        count+=1
                    if count==size:
                        break
            if len(hrec)>=num:
                break
        #print idxmap[t[1]],len(hrec)
    l=list(hrec)
    shuffle(l)
    rec[hacker]=l
    iter+=1
#print rec

user 9999:cd48fcb1f422ead5


In [23]:
import os
os.remove("recommendation.csv")
f = open('recommendation.csv','w')
for key in rec:
    ans=''
    l=rec[key]
    l=l[0:10]
    shuffle(l)
    for c in l:
        ans+=','+c    
    f.write(key+ans+'\n')    
f.close()